# The CUTHI code

In [1]:
def cuthi(x, y, stations, s=0.5, alpha=2):
    
    # Check for exact match first (avoiding all the loops if found)
    idx = np.where((stations[:, 0] == x) & (stations[:, 1] == y))
    if idx[0].size > 0:
        return stations[idx[0][0], 2]
    
    # Precompute distances between all pairs of stations
    dx = stations[:, 0].reshape(-1, 1) - stations[:, 0]
    dy = stations[:, 1].reshape(-1, 1) - stations[:, 1]
    dist2 = dx**2 + dy**2  # r^2 (distance squared)
    
    wtotal = 0
    w = np.zeros(len(stations))
    
    for k in range(len(stations)):
        r2 = dist2[k, k]
        if r2 != 0:
            w[k] = 1. / r2**(alpha / 2)
            wcuthi = 1.
            
            # Precompute distance terms for all stations relative to station k
            adist2 = dist2[k, :]
            bdist2 = dist2[:, k]
            cosgama = np.zeros(len(stations))
            nangle = np.zeros(len(stations))
            
            for m in range(len(stations)):
                if adist2[m] > 0 and bdist2[m] > 0:
                    cosgama[m] = (adist2[m] + bdist2[m] - r2) / (2.0 * np.sqrt(adist2[m]) * np.sqrt(bdist2[m]))
                    nangle[m] = (cosgama[m] + 1.) / 2.
                    nangle[m] = max(0., nangle[m])  # Ensuring nangle is non-negative
                    
                wcuthi *= nangle[m]**s
            
            w[k] *= wcuthi
            wtotal += w[k]
    
    res = 0
    if wtotal == 0:
        return 0  # Or some fallback value if no valid weight is computed
    
    for k in range(len(stations)):
        r2 = dist2[k, k]
        if r2 == 0:
            res = stations[k, 2]
        else:
            w[k] = 1. / r2**(alpha / 2)
            wcuthi = 1.
            
            # Precompute distance terms for all stations relative to station k
            for m in range(len(stations)):
                adist2 = dist2[k, m]
                bdist2 = dist2[m, k]
                if adist2 > 0 and bdist2 > 0:
                    cosgama = (adist2 + bdist2 - r2) / (2.0 * np.sqrt(adist2) * np.sqrt(bdist2))
                    nangle = (cosgama + 1.) / 2.
                    nangle = max(0., nangle)  # Ensuring nangle is non-negative
                    
                wcuthi *= nangle**s
            
            w[k] *= wcuthi
            res += stations[k, 2] * w[k] / wtotal
    
    return res

# Cross-validation code

In [2]:
import sklearn.metrics
def crossvalidation(stations, alpha = 2):
    size=len(stations)
    r2 = 0
    resultcuthi = []
    for i in range(size):
            cv = []
            for j in range(size):
                if i!=j:
                   cv.append(stations[j])                      
            cv = np.asarray(cv)
            resultcuthi.append(cuthi(stations[i,0],stations[i,1],cv, s=0.5, alpha=alpha))
    r2   += sklearn.metrics.r2_score(stations[:,2], resultcuthi)
    return r2

# Build the stations data #

In [5]:
import random 
import numpy as np
def getFunction(size):
    minlat = -50
    maxlat = 150
    minlng = -50
    maxlng = 150
    
    stations = []
    for i in range (size):
        latrand = random.uniform(minlat, maxlat)
        lngrand = random.uniform(minlng, maxlng)
        z = latrand **2 - lngrand **2
        stations.append([lngrand, latrand, z])
    stations = np.asarray(stations)
    
    return stations

# Main code
1. build the station data (using mathematical function)
2. calculate the CUTHI accuracy using cross-validation (optional) - it is better to use interpolation only if the score is positive
3. interpolation

In [7]:
size=60
stations = getFunction(size)
print("Cross validation score is:",crossvalidation(stations), " should be positive")
resultcuthi = cuthi(130,130,stations)
print("Interpolated value of x=0, y=0 (should be 0, maximum value is 22500) is:", cuthi(0,0,stations))

Cross validation score is: 0.9576993634824083  should be positive
Interpolated value of x=0, y=0 (should be 0, maximum value is 22500) is: 111.13697178888864


# More codes #
Cuthi's code above is much faster than the following code, but the following code is simpler to read, both give the same prediction. 
The following code is an Inverse Distance Weighting (IDW) code.

In [1]:
def readable_cuthi(x,y,stations, s=0.5, alpha=2):
    for i in range(len(stations)):
        if stations[i][0] == x and stations[i][1] == y:
            return stations[i][2]

    wtotal=0
    for k in range(len(stations)):
        r = ((stations[k,0]-x)**2+(stations[k,1]-y)**2)  # it's r^2
        if r!=0:
            w=1./r**(alpha/2)
            wcuthi = 1.
            for m in range(len(stations)):
                adist2 = float((stations[k][0] - stations[m][0]) ** 2 + (stations[k][1] - stations[m][1]) ** 2)
                bdist2 = float((stations[m][0] - x) ** 2 + (stations[m][1] - y) ** 2)
                if adist2>0 and bdist2>0:
                    cosgama = (adist2 + bdist2 - r) / (2.0 * adist2**.5 * bdist2**.5)
                    nangle = ((cosgama+1.)/2.)
                    if nangle<0:
                        if nangle<-0.01:
                            print ('nangle ngnalge')
                        else:
                            nangle = 0.
                            
                    wcuthi*=nangle**s
            w = w*wcuthi        
            wtotal+=w
    res=0
    for k in range(len(stations)):
        r = ((stations[k,0]-x)**2+(stations[k,1]-y)**2)
        if r == 0:
            res = stations[k,2]
        else:
            w=1./r**(alpha/2)
            wcuthi=1.
            for m in range(len(stations)):
                adist2 = float((stations[k][0] - stations[m][0]) ** 2 + (stations[k][1] - stations[m][1]) ** 2)
                bdist2 = float((stations[m][0] - x) ** 2 + (stations[m][1] - y) ** 2)
                if adist2>0 and bdist2>0:
                    cosgama = (adist2 + bdist2 - r) / (2.0 * adist2**.5 * bdist2**.5)
                    nangle = ((cosgama+1.)/2.)
                    if nangle<0:
                        if nangle<-0.01:
                            print ('nangle ngnalge')
                        else:
                            nangle = 0.
                    wcuthi*=nangle**s
            w = w*wcuthi
            res+=stations[k,2]*w/wtotal
    
    return res

In [ ]:
def idw(x, y, stations, alpha=2):
   
    # Check for exact match first (avoiding all the loops if found)
    idx = np.where((stations[:, 0] == x) & (stations[:, 1] == y))
    if idx[0].size > 0:
        return stations[idx[0][0], 2]
    
    # Precompute distances between all stations and the point (x, y)
    dx = stations[:, 0] - x
    dy = stations[:, 1] - y
    r2 = dx**2 + dy**2  # r^2
    
    # Calculate total weight
    wtotal = np.sum(1. / r2**(alpha / 2))
    
    # If wtotal is 0, return 0 (or handle this case according to your needs)
    if wtotal == 0:
        return 0
    
    # Weighted sum of station values
    w = 1. / r2**(alpha / 2)
    res = np.sum(stations[:, 2] * w / wtotal)
    
    return res